In [1]:
print("Ok")

Ok


In [2]:
%pwd

'd:\\EPICS2.0\\AshaAI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\EPICS2.0\\AshaAI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader # type: ignore
from langchain.text_splitter import RecursiveCharacterTextSplitter # type: ignore

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
# extracted_data

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Lengath of Text Chunks", len(text_chunks))

Lengath of Text Chunks 5860


In [11]:
# text_chunks


In [12]:
from langchain_huggingface import HuggingFaceEmbeddings # type: ignore

In [13]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [14]:
embeddings = download_hugging_face_embeddings()

c:\Users\anshu\anaconda3\envs\ashaai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [16]:
# query_result

In [17]:
from dotenv import load_dotenv # type: ignore
load_dotenv()

True

In [18]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
TOGETHER_API_KEY=os.environ.get('TOGETHER_API_KEY')

In [24]:
from pinecone.grpc import PineconeGRPC as Pinecone # type: ignore
from pinecone import ServerlessSpec # type: ignore
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "ashaai"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "ashaai",
    "metric": "cosine",
    "host": "ashaai-4345at0.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [25]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["TOGETHER_API_KEY"] = TOGETHER_API_KEY

In [26]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore # type: ignore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [27]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore # type: ignore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [28]:
docsearch


In [29]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [30]:
retrieved_docs = retriever.invoke("what is Acromegaly?")


In [31]:
retrieved_docs

[Document(id='e89361a2-23f9-4610-a900-85979e8823fb', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 45.0, 'page_label': '46', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='Whitehouse Station, NJ: Merck Research Laboratories,\n1997.\nLarsen, D. E., ed. Mayo Clinic Family Health Book.New York:\nWilliam Morrow and Co., Inc., 1996.\nJohn T. Lohr, PhD\nAcromegaly and gigantism\nDefinition\nAcromegaly is a disorder in which the abnormal\nrelease of a particular chemical from the pituitary gland\nin the brain causes increased growth in bone and soft tis-\nsue, as well as a variety of other disturbances throughout\nthe body. This chemical released from the pituitary gland'),
 Document(id='df6b070d-e71a-4e5d-acca-b5f09da1e043', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 46.0,

In [32]:
from langchain.llms import Together # type: ignore

llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.1",  # ✅ Working model
    temperature=0.4,
    max_tokens=500
)


C:\Users\anshu\AppData\Local\Temp\ipykernel_11484\24720287.py:3: LangChainDeprecationWarning: The class `Together` was deprecated in LangChain 0.0.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-together package and should be used instead. To use it run `pip install -U :class:`~langchain-together` and import as `from :class:`~langchain_together import Together``.
  llm = Together(


In [33]:
from langchain.llms import Together # type: ignore
from langchain.chains import create_retrieval_chain # type: ignore
from langchain.chains.combine_documents import create_stuff_documents_chain # type: ignore
from langchain_core.prompts import ChatPromptTemplate # type: ignore
from langchain_core.runnables import Runnable # type: ignore
import os

# Set Together API key
os.environ["TOGETHER_API_KEY"] = "8c2f5d82e8f6608273adbe1cb4c151589073baf128eccf959c1bb8936b88d92b"  # Replace with your actual API key

# Initialize Together LLM
llm = Together(
    model="mistralai/Mistral-7B-Instruct-v0.1",
    temperature=0.4,
    max_tokens=500
)

# System prompt template
system_prompt = (
    "You are AshaAI, a helpful and concise medical assistant designed for Indian farmers. "
    "Use the following retrieved context from trusted medical sources to answer the user's query. "
    "If the answer is unknown, say so honestly. Keep the answer short (2-3 sentences), friendly, and informative.\n\n"
    "{context}"
)

# Define the ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [34]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [35]:
response = rag_chain.invoke({"input": "what is Acromegaly?"})
print(response["answer"])




Acromegaly is caused by overproduction of growth hormone (GH) by the pituitary gland. This overproduction causes increased growth in bone and soft tissue, as well as a variety of other disturbances throughout the body.

Symptoms
Acromegaly is characterized by a gradual increase in height, weight gain, and an enlargement of the hands and feet. The enlargement of the hands and feet can be so severe that they become deformed and difficult to walk. Other symptoms may include a deepened voice, thickening of the skin, oily skin, joint pain, and difficulty sleeping.

Diagnosis
Diagnosis of acromegaly is based on a combination of clinical symptoms, laboratory tests, and imaging studies. The primary diagnostic test for acromegaly is a laboratory test that measures the level of insulin-like growth factor-1 (IGF-1) in the blood. IGF-1 levels are typically elevated in acromegaly.

Treatment
The treatment of acromegaly is aimed at reducing the production of GH by the pituitary gland. This can be 

In [36]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])



AshaAI: Stats, in the context of sports, refers to the collection and analysis of data related to various aspects of a sport, such as performance metrics, player statistics, and team performance. This data can be used to inform training and coaching strategies, as well as to evaluate the effectiveness of different tactics and strategies. Stats can also be used to identify trends and patterns in player performance and team performance, which can help teams make more informed decisions about their strategies and tactics.
